## Lezione 5

**Agenda**
+ ripasso e completamento RNN
+ Transformers

**Note**
File con Risorse Utili (del prof.): lo condividerà

### RNN

In [ ]:
Understanding LSTM and GRU RNN
https://colah.github.io/posts/2015-08-Understanding-LSTMs/

#### Strati Ricorrenti

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

2023-09-20 09:18:09.416161: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-20 09:18:10.154352: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-20 09:18:10.162982: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-20 09:18:13.794600: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from keras.layers import GRU

In [3]:
# GRU Layer
gru = GRU(units = 128) # UNITS: dimensione hidden state (numero di float del vettore hidden state!)

In [ ]:
# Tensori in Input e in Output
INPUT = batch_size x seq_len x embedding_dim
OUT = batch_size x hidden_units
# vedi appunti

In [6]:
# Input random
x = tf.random.normal(shape=[32, 20, 256])

In [7]:
y = gru(x)

In [9]:
y.shape

TensorShape([32, 128])

#### Strati di GRU

In [11]:
# Strati
gru1 = GRU(units = 128, return_sequences=True)
gru2 = GRU(units = 128, return_sequences=True)
gru3 = GRU(units = 128, return_sequences=True)
gru4 = GRU(units = 128) # ultimo strato/cella -> output (in questo caso non serve return_sequences=True nell'output)
# di solito il numero di units resta uguale

# input
x = tf.random.normal(shape=[32, 20, 256])

# elaborazione
y1 = gru1(x)
y2 = gru2(y1)
y3 = gru3(y2)
y4 = gru4(y3)

TensorShape([32, 128])

In [13]:
# shape
print(y4.shape,
y1.shape)

(32, 128) (32, 20, 128)


#### LSTM

In [14]:
# l'unica differenza: hanno un doppio stato
from keras.layers import LSTM

In [16]:
lstm = LSTM(units=128)
y = lstm(x)
# output come prima, perché in OUTPUT c'è, anche qui, un hidden state
y.shape

TensorShape([32, 128])

In [17]:
lstm = LSTM(units=128, return_sequences=True)
y = lstm(x)
# output come prima, 1 hidden state per ogni parola
y.shape

TensorShape([32, 20, 128])

##### LSTM con Return_State

In [26]:
lstm = LSTM(units=128, return_sequences=True, return_state=True)
output, hidden_state, cell_state = lstm(x)

In [30]:
print([x.shape for x in [output, hidden_state, cell_state]])
# cerca differenza sono output e hidden_state

[TensorShape([32, 20, 128]), TensorShape([32, 128]), TensorShape([32, 128])]


#### GRU con Return State
output + stato interno (cell state)

In [23]:
gru = GRU(units = 128, return_sequences=True, return_state=True)
output, state = gru(x)

In [ ]:
output
state
# qui non si nota, torna sulle LSTM

### RNN Bidirezionale

In [32]:
from keras.layers import Bidirectional

In [33]:
lstm = LSTM(units=128, return_sequences=True)
bidirectional = Bidirectional(lstm)

In [35]:
bidirectional(x).shape # 256 perché legge due volte:
# una rappresentazione più ricca: alla luce del contesto destro e del contesto sx

TensorShape([32, 20, 256])

### Encoder-Decoder

In [ ]:
Vedi Appunti per Teoria

#### Tokenizzazione WordPiece

In [36]:
!pip install tokenizers # di HuggingFace

In [37]:
!wget https://www.manythings.org/anki/ita-eng.zip
!unzip ita-eng.zip

--2023-09-20 11:26:37--  https://www.manythings.org/anki/ita-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8220355 (7.8M) [application/zip]
Saving to: ‘ita-eng.zip’

ita-eng.zip         100%[===================>]   7.84M  2.89MB/s    in 2.7s    

2023-09-20 11:26:41 (2.89 MB/s) - ‘ita-eng.zip’ saved [8220355/8220355]

Archive:  ita-eng.zip
  inflating: ita.txt                 
  inflating: _about.txt              


In [39]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece

In [40]:
wp_tokenizer = Tokenizer(model = WordPiece(unk_token="[UNK]") ) 
# model = il modello di tokenizzazione

Tokenizzazione, ha 4 step, (vedi appunti)
+ normalizzazione (ex. Unicode_Normalization: https://unicode.org/reports/tr15/)
+ Pre-TOkenizzazione
+ TOkenizzazione
+ Post-Processing

see https://platform.openai.com/tokenizer
and https://huggingface.co/docs

In [45]:
from tokenizers.normalizers import BertNormalizer
from tokenizers.pre_tokenizers import BertPreTokenizer
# post process?
from tokenizers.trainers import WordPieceTrainer

In [44]:
# WordPiece Tokenizer
wp_tokenizer.normalizer = BertNormalizer()
wp_tokenizer.pre_tokenizer = BertPreTokenizer()
# wp_tokenizer.post_process = 

In [46]:
def lang_iterator(lang = "en"):
    with open("ita.txt", "r") as f:
        while line := f.readline()[:-1]:
            en_sent, it_sent, _ = line.split("\t")
            yield en_sent if lang =="en" else it_sent

In [48]:
special_tokens = ["[PAD]", "[UNK]", "[BOS]", "[EOS]" ]

wp_tokenizer.train_from_iterator(
    lang_iterator("en"), # iteratore con la sequenza di parole/token
    trainer = WordPieceTrainer(vocab_size = 10_000, special_tokens=special_tokens) 
)

In [53]:
wp_tokenizer.get_vocab()
# se è scritto ##where, means: token agganciato a sth prima

{'##edient': 5118,
 'kidding': 3132,
 'dev': 3489,
 'go': 150,
 'confusing': 5654,
 'skept': 2874,
 '##wheel': 8339,
 'haw': 4615,
 '##ully': 1787,
 'beach': 1850,
 'hope': 624,
 '##asted': 5125,
 'adjusted': 8359,
 'goal': 3222,
 '##po': 6038,
 'glass': 1084,
 'stuck': 2566,
 'favorite': 859,
 'trying': 995,
 'med': 3022,
 'sha': 4754,
 'domestic': 6763,
 'excuse': 1774,
 'strugg': 6204,
 'this': 158,
 'plates': 6477,
 '##ither': 1674,
 'master': 4816,
 'celebrate': 3414,
 'wheel': 5548,
 'rust': 7605,
 'asked': 583,
 'stu': 310,
 '##market': 3212,
 'oranges': 3588,
 'violent': 4911,
 'seldom': 2585,
 '##chief': 7424,
 'substit': 8564,
 'john': 1416,
 'dice': 9088,
 'washing': 2622,
 'gunsh': 7976,
 'members': 5022,
 'could': 384,
 'masks': 8634,
 'cartwheel': 8347,
 'scored': 5744,
 'escap': 2045,
 'handle': 2867,
 'dipl': 3847,
 'encourage': 5775,
 'examine': 6521,
 '##estern': 8457,
 'slippery': 5796,
 '##omes': 7627,
 'healthy': 2509,
 'cont': 723,
 'presence': 5483,
 'toss': 5929

In [56]:
# Un esempio con una frase custom
encoded = wp_tokenizer.encode("I lost my pen somewhere")
encoded.ids # 5 int, 5 token (indici nella matrice di embedding)

[37, 614, 169, 775, 1974]

In [57]:
encoded.tokens

['i', 'lost', 'my', 'pen', 'somewhere']

In [59]:
encoded = wp_tokenizer.encode("I am encountering my idol")
encoded.ids, encoded.tokens

([37, 386, 3100, 755, 5117, 169, 644, 351],
 ['i', 'am', 'encou', '##nt', '##ering', 'my', 'id', '##ol'])

#### BPE

In [66]:
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace, Punctuation, ByteLevel
from tokenizers.trainers import BpeTrainer

In [67]:
bpe_tokenizer = Tokenizer(model = BPE(unk_token="[UNK]") ) 

In [68]:
# BPE Tokenizer
bpe_tokenizer.normalizer = BertNormalizer()
bpe_tokenizer.pre_tokenizer = ByteLevel()

def lang_iterator(lang = "en"):
    with open("ita.txt", "r") as f:
        while line := f.readline()[:-1]:
            en_sent, it_sent, _ = line.split("\t")
            yield en_sent if lang =="en" else it_sent

special_tokens = ["[PAD]", "[UNK]", "[BOS]", "[EOS]" ]

bpe_tokenizer.train_from_iterator(
    lang_iterator("en"), # iteratore con la sequenza di parole/token
    trainer = BpeTrainer(vocab_size = 10_000, special_tokens=special_tokens) 
)

In [69]:
encoded = bpe_tokenizer.encode("I am encountering my idol")
encoded.ids, encoded.tokens
# G con il puntino è o space o inizio frase

([65, 388, 2990, 669, 7983, 158, 1894, 354],
 ['Ġi', 'Ġam', 'Ġenc', 'oun', 'tering', 'Ġmy', 'Ġid', 'ol'])

#### Note

In [73]:
from tokenizers import BertWordPieceTokenizer
# è già un tokenizer completo

In [75]:
bwp_tokenizer = BertWordPieceTokenizer()
bwp_tokenizer.train_from_iterator(lang_iterator("en"), vocab_size=10_000)

In [78]:
encoded = bwp_tokenizer.encode("I was in Rome")
encoded.tokens

['i', 'was', 'in', 'rome']

In [ ]:
encoded.ids # è il vettore che rappresenta la frase/sequenza, ogni id corrisponde a una riga della matrce di embedding (ovvero è una parola))

## PyTorch
Cenni di PyTorch perché sono le 16:49

In [ ]:
vedi altro Notebook